In [18]:
!pip install !pip install tensorflow_io
!pip install keras-tuner
!pip install print_schema
!pip install pydub
!pip install opensmile
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Invalid requirement: '!pip'
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached keras_tuner-1.1.3-py3-none-any.whl (135 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux

In [19]:
import pandas as pd
import numpy as np
import torch
import gc

In [20]:
df = pd.read_csv("train.csv")
texts = list(df.loc[:,"Content"])
labels = list(df.loc[:,"Label"])

In [21]:
df = pd.read_csv("test.csv")
df["Content"].fillna("",inplace=True)
test_texts = list(df.loc[:,"Content"])
test_labels = list(df.loc[:,"Label"])

In [22]:
train_texts = texts[: 133]
train_labels = labels[: 133]
val_texts = texts[133:]
val_labels = labels[133:]

In [23]:
import torch
import torchvision
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /home/jupyter/.cache/torch/hub/huggingface_pytorch-transformers_main


In [24]:
class ADdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [25]:
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 256, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            
            nn.Linear(D_in, 256),
            nn.BatchNorm1d(num_features=256), ## kk
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            #nn.BatchNorm1d(num_features=128),
            #nn.ReLU(),
            #nn.Linear(128, 1)
            
            #nn.BatchNorm1d(num_features=128), ###
            #nn.ReLU(),
            
            #nn.Dropout(0.5),

            #nn.Linear(H, D_out)
            #nn.Linear(128, 2)
            #nn.Linear(128, 1)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader


def initialize_model(epochs=50):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_loader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [27]:
import random
import time

# Specify loss function
#loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.BCELoss()
loss_fn =  torch.nn.BCEWithLogitsLoss()
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_loader, optim, val_loader=None, epochs=50, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = [] 
    for epoch_i in range(epochs):
        train_loss_sum = 0
        train_accuracy_epoch = 0
        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for batch in (train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Perform a forward pass. This will return logits.
            logits = model(input_ids, attention_mask)

            # Compute loss and accumulate the loss values
            logits = logits.reshape(-1) #silebilirsin
            
            loss = loss_fn(logits, labels.float())
            train_loss_sum += loss.item()

            logits_class = logits > 0.5
            train_acc = (labels == logits_class).sum().item() / labels.size(0)
            train_accuracy_epoch += train_acc

            # Perform a backward pass to calculate gradients
            loss.backward()


            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

        avg_train_loss = np.round(train_loss_sum/len(train_loader),2)
        avg_train_acc = np.round(train_accuracy_epoch/len(train_loader),2)

        if evaluation == True:
            avg_val_loss, avg_val_acc = evaluate(model, val_loader)
        print('Epoch {}, train loss {} , val loss is {}, train acc is {}, val acc is {} '.format(epoch_i,avg_train_loss,avg_val_loss,avg_train_acc,avg_val_acc))
        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        train_acc_list.append(avg_train_acc)

    print("Training complete!")
    return model,train_loss_list,val_loss_list,train_acc_list,val_acc_list


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_loss_sum = 0
    val_accuracy_epoch = 0
    # For each batch in our validation set...
    for batch in val_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        val_loss_sum +=loss.item()
        avg_val_loss = np.round(val_loss_sum/len(val_dataloader),2)

        logits_class = logits > 0.5
        val_acc = (labels == logits_class).sum().item() / labels.size(0)
        val_accuracy_epoch += val_acc
        avg_val_acc = np.round(val_accuracy_epoch/len(val_dataloader),2)

    return avg_val_loss, avg_val_acc


def evaluate_test(model, test_dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    test_loss_sum = 0
    test_accuracy_epoch = 0
    predictions = []
    labels_list = []
    for batch in test_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)
        labels_list.append(labels.cpu())

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        test_loss_sum +=loss.item()
        avg_test_loss = np.round(test_loss_sum/len(test_dataloader),5)

        logits_class = logits > 0.5
        predictions.append(logits.cpu())
        test_acc = (labels == logits_class).sum().item() / labels.size(0)
        test_accuracy_epoch += test_acc
        avg_test_acc = np.round(test_accuracy_epoch/len(test_dataloader),5)

    return avg_test_loss, avg_test_acc, predictions, labels_list

In [28]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
train_dataset = ADdataset(train_encodings, train_labels)
val_dataset = ADdataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = ADdataset(test_encodings, test_labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [29]:
seeds = list(range(0,60,6))
mean_val_acc = {}
last_val_acc = {}
test_acc = {}

In [30]:
for i in seeds:
  set_seed(i)    # Set seed for reproducibility
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  mean_val_acc[i] = []
  last_val_acc[i] = []
  test_acc[i] = []
  for _ in range(5):
    bert_classifier, optimizer, scheduler = initialize_model(epochs=20)
    bert_classifier,train_loss_list,val_loss_list,train_acc_list,val_acc_list = train(bert_classifier, train_loader, optimizer, val_loader, epochs=20, evaluation=True)
    test_loss, test_accuracy, predictions, labels_l = evaluate_test(bert_classifier, test_loader)
    print("Test Loss: {}, Test Accuracy: {}".format(test_loss,test_accuracy))
    mean_val_acc[i].append(np.mean(val_acc_list))
    last_val_acc[i].append(val_acc_list[-1])
    test_acc[i].append(test_accuracy)
    gc.collect()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

Epoch 0, train loss 0.6 , val loss is 0.61, train acc is 0.64, val acc is 0.77 
Epoch 1, train loss 0.45 , val loss is 0.48, train acc is 0.78, val acc is 0.83 
Epoch 2, train loss 0.28 , val loss is 0.39, train acc is 0.89, val acc is 0.81 
Epoch 3, train loss 0.17 , val loss is 0.44, train acc is 0.95, val acc is 0.88 
Epoch 4, train loss 0.08 , val loss is 1.06, train acc is 0.99, val acc is 0.58 
Epoch 5, train loss 0.05 , val loss is 0.96, train acc is 1.0, val acc is 0.73 
Epoch 6, train loss 0.04 , val loss is 0.5, train acc is 0.99, val acc is 0.85 
Epoch 7, train loss 0.08 , val loss is 0.67, train acc is 0.95, val acc is 0.79 
Epoch 8, train loss 0.03 , val loss is 0.6, train acc is 1.0, val acc is 0.81 
Epoch 9, train loss 0.02 , val loss is 0.62, train acc is 1.0, val acc is 0.81 
Epoch 10, train loss 0.08 , val loss is 2.33, train acc is 0.91, val acc is 0.5 
Epoch 11, train loss 0.03 , val loss is 0.65, train acc is 1.0, val acc is 0.83 
Epoch 12, train

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

Epoch 0, train loss 0.63 , val loss is 0.58, train acc is 0.53, val acc is 0.77 
Epoch 1, train loss 0.42 , val loss is 0.52, train acc is 0.74, val acc is 0.83 
Epoch 2, train loss 0.3 , val loss is 0.4, train acc is 0.85, val acc is 0.88 
Epoch 3, train loss 0.16 , val loss is 0.39, train acc is 0.94, val acc is 0.85 
Epoch 4, train loss 0.07 , val loss is 0.62, train acc is 0.99, val acc is 0.83 
Epoch 5, train loss 0.05 , val loss is 0.66, train acc is 1.0, val acc is 0.83 
Epoch 6, train loss 0.04 , val loss is 0.63, train acc is 1.0, val acc is 0.83 
Epoch 7, train loss 0.04 , val loss is 0.57, train acc is 1.0, val acc is 0.85 
Epoch 8, train loss 0.04 , val loss is 0.57, train acc is 1.0, val acc is 0.85 
Epoch 9, train loss 0.03 , val loss is 0.53, train acc is 1.0, val acc is 0.85 
Epoch 10, train loss 0.03 , val loss is 0.73, train acc is 1.0, val acc is 0.81 
Epoch 11, train loss 0.03 , val loss is 0.52, train acc is 1.0, val acc is 0.85 
Epoch 12, train 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Start training...

Epoch 0, train loss 0.63 , val loss is 0.63, train acc is 0.56, val acc is 0.81 
Epoch 1, train loss 0.42 , val loss is 0.42, train acc is 0.84, val acc is 0.83 
Epoch 2, train loss 0.27 , val loss is 1.2, train acc is 0.92, val acc is 0.33 
Epoch 3, train loss 0.19 , val loss is 0.47, train acc is 0.9, val acc is 0.83 
Epoch 4, train loss 0.07 , val loss is 0.32, train acc is 0.99, val acc is 0.92 
Epoch 5, train loss 0.06 , val loss is 0.49, train acc is 0.99, val acc is 0.83 
Epoch 6, train loss 0.04 , val loss is 1.23, train acc is 1.0, val acc is 0.52 
Epoch 7, train loss 0.03 , val loss is 0.61, train acc is 1.0, val acc is 0.81 
Epoch 8, train loss 0.04 , val loss is 1.69, train acc is 1.0, val acc is 0.52 
Epoch 9, train loss 0.04 , val loss is 1.71, train acc is 1.0, val acc is 0.52 
Epoch 10, train loss 0.04 , val loss is 0.6, train acc is 1.0, val acc is 0.83 
Epoch 11, train loss 0.03 , val loss is 0.62, train acc is 1.0, val acc is 0.83 
Epoch 12, train 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

Epoch 0, train loss 0.59 , val loss is 0.71, train acc is 0.62, val acc is 0.46 
Epoch 1, train loss 0.39 , val loss is 0.47, train acc is 0.83, val acc is 0.85 
Epoch 2, train loss 0.24 , val loss is 0.35, train acc is 0.92, val acc is 0.9 
Epoch 3, train loss 0.11 , val loss is 0.46, train acc is 0.97, val acc is 0.83 
Epoch 4, train loss 0.15 , val loss is 0.63, train acc is 0.97, val acc is 0.83 
Epoch 5, train loss 0.13 , val loss is 1.55, train acc is 0.9, val acc is 0.56 
Epoch 6, train loss 0.05 , val loss is 0.47, train acc is 1.0, val acc is 0.88 
Epoch 7, train loss 0.03 , val loss is 1.92, train acc is 1.0, val acc is 0.54 
Epoch 8, train loss 0.03 , val loss is 0.55, train acc is 1.0, val acc is 0.88 
Epoch 9, train loss 0.04 , val loss is 0.59, train acc is 1.0, val acc is 0.85 
Epoch 10, train loss 0.04 , val loss is 0.59, train acc is 1.0, val acc is 0.85 
Epoch 11, train loss 0.02 , val loss is 0.57, train acc is 1.0, val acc is 0.85 
Epoch 12, train

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

Epoch 0, train loss 0.63 , val loss is 0.63, train acc is 0.6, val acc is 0.88 
Epoch 1, train loss 0.53 , val loss is 0.42, train acc is 0.73, val acc is 0.79 
Epoch 2, train loss 0.34 , val loss is 0.5, train acc is 0.87, val acc is 0.85 
Epoch 3, train loss 0.16 , val loss is 0.35, train acc is 0.95, val acc is 0.9 
Epoch 4, train loss 0.07 , val loss is 0.55, train acc is 0.99, val acc is 0.83 
Epoch 5, train loss 0.08 , val loss is 0.57, train acc is 0.99, val acc is 0.83 
Epoch 6, train loss 0.03 , val loss is 0.45, train acc is 1.0, val acc is 0.85 
Epoch 7, train loss 0.04 , val loss is 0.5, train acc is 1.0, val acc is 0.85 
Epoch 8, train loss 0.03 , val loss is 0.39, train acc is 1.0, val acc is 0.9 
Epoch 9, train loss 0.03 , val loss is 0.65, train acc is 1.0, val acc is 0.88 
Epoch 10, train loss 0.03 , val loss is 0.59, train acc is 1.0, val acc is 0.85 
Epoch 11, train loss 0.09 , val loss is 0.58, train acc is 0.93, val acc is 0.85 
Epoch 12, train l

In [31]:
print(mean_val_acc)
print(last_val_acc)
print(test_acc)

{0: [0.7390000000000001, 0.782, 0.726, 0.6980000000000001, 0.734], 6: [0.8245000000000001, 0.7770000000000001, 0.7545, 0.7885, 0.72], 12: [0.763, 0.7474999999999999, 0.7665, 0.7619999999999999, 0.6690000000000002], 18: [0.717, 0.7800000000000001, 0.7635, 0.7350000000000001, 0.7205], 24: [0.7715000000000001, 0.7775, 0.7565000000000001, 0.7474999999999999, 0.739], 30: [0.7150000000000001, 0.716, 0.7785, 0.7715, 0.7224999999999999], 36: [0.8240000000000002, 0.7625, 0.7314999999999999, 0.8025, 0.7535000000000001], 42: [0.712, 0.7515000000000002, 0.7290000000000001, 0.7304999999999999, 0.7705], 48: [0.772, 0.771, 0.7615, 0.7715, 0.775], 54: [0.8385000000000001, 0.7605000000000002, 0.7280000000000001, 0.766, 0.8035]}
{0: [0.81, 0.54, 0.52, 0.81, 0.5], 6: [0.88, 0.85, 0.5, 0.85, 0.5], 12: [0.83, 0.52, 0.83, 0.85, 0.48], 18: [0.83, 0.88, 0.54, 0.81, 0.85], 24: [0.81, 0.52, 0.83, 0.79, 0.85], 30: [0.5, 0.54, 0.85, 0.83, 0.83], 36: [0.88, 0.83, 0.48, 0.83, 0.85], 42: [0.77, 0.56, 0.54, 0.85, 0.8

In [32]:
import csv
with open('/content/gdrive/MyDrive/mean_val_acc.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
  w = csv.DictWriter(f, mean_val_acc.keys())
  w.writeheader()
  w.writerow(mean_val_acc)
  f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/mean_val_acc.csv'

In [ ]:
with open('/content/gdrive/MyDrive/last_val_acc.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
  w = csv.DictWriter(f, last_val_acc.keys())
  w.writeheader()
  w.writerow(last_val_acc)
  f.close()

In [ ]:
with open('/content/gdrive/MyDrive/test_acc.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
  w = csv.DictWriter(f, test_acc.keys())
  w.writeheader()
  w.writerow(test_acc)
  f.close()